# **<mark>Indexes</mark>**

Chỉ mục là cấu trúc dữ liệu đặc biệt được liên kết với bảng hoặc dạng xem giúp tăng tốc truy vấn. SQL Server cung cấp hai loại chỉ mục: chỉ mục theo cụm và chỉ mục không được phân cụm.

## **<mark>Clustered Indexes</mark>**

In [ ]:
CREATE TABLE production.parts(
    part_id   INT NOT NULL, 
    part_name VARCHAR(100)
)

In [ ]:
INSERT INTO 
    production.parts(part_id, part_name)
VALUES
    (1,'Frame'),
    (2,'Head Tube'),
    (3,'Handlebar Grip'),
    (4,'Shock Absorber'),
    (5,'Fork')

Bảng production.parts không có khóa chính (primary key). Do đó, SQL Server lưu trữ các hàng của nó trong một cấu trúc không có thứ tự được gọi là heap.

  

Khi bạn truy vấn dữ liệu từ bảng production.parts, trình tối ưu hóa truy vấn (query optimizer) cần quét toàn bộ bảng để tìm kiếm.

In [ ]:
SELECT 
    part_id, 
    part_name
FROM 
    production.parts
WHERE 
    part_id = 5

Nếu bạn hiển thị kế hoạch thực thi ước tính (execution plan) trong SQL Server Management Studio, bạn sẽ thấy cách SQL Server đưa ra kế hoạch truy vấn sau:

![indexes-1](.\SQL-Server-Clustered-Index-Table-Scan-1.png)

Lưu ý rằng để hiển thị kế hoạch thực thi ước tính (execution plan) trong SQL Server Management Studio, bạn nhấp vào nút Hiển thị kế hoạch thực thi ước tính hoặc chọn truy vấn và nhấn phím tắt Ctrl + L:

![indexes-2](.\SQL-Server-Display-Estimated-Execution-Plan.png)

Bởi vì bảng production.parts chỉ có năm hàng, truy vấn thực thi rất nhanh. Tuy nhiên, nếu bảng chứa nhiều hàng, sẽ mất nhiều thời gian và tài nguyên để tìm kiếm dữ liệu.

Để giải quyết vấn đề này, SQL Server cung cấp một cấu trúc chuyên dụng để tăng tốc độ truy xuất các hàng từ một bảng được gọi là chỉ mục.

SQL Server có hai loại chỉ mục: chỉ mục theo cụm (clustered indexes) và chỉ mục không được phân cụm (non-clustered indexes).

Một chỉ mục được phân cụm lưu trữ các hàng dữ liệu trong một cấu trúc được sắp xếp dựa trên các giá trị chính của nó. Mỗi bảng chỉ có một chỉ mục nhóm vì các hàng dữ liệu chỉ có thể được sắp xếp theo một thứ tự. Một bảng có chỉ mục được phân cụm được gọi là bảng được phân nhóm.

![indexes-3](.\SQL-Server-Clustered-B-Tree.png)

Chỉ mục được phân nhóm tổ chức dữ liệu bằng cách sử dụng một cây có cấu trúc đặc biệt được gọi là B-tree (hoặc cây cân bằng - balanced tree) cho phép tìm kiếm, chèn, cập nhật và xóa theo thời gian phân bổ theo logarit

Trong cấu trúc này, nút trên cùng của B-tree được gọi là nút gốc (root node). Các nút ở mức dưới cùng được gọi là các nút lá (leaf nodes). Bất kỳ mức chỉ mục nào giữa nút gốc và nút lá được gọi là mức trung gian (intermediate nodes).

Trong B-Tree, nút gốc và các nút cấp trung gian chứa các trang chỉ mục chứa các hàng chỉ mục. Các nút lá chứa các trang dữ liệu của bảng bên dưới. Các trang trong mỗi cấp của chỉ mục được liên kết bằng cách sử dụng một cấu trúc khác được gọi là danh sách liên kết kép (doubly-linked list).

## **Chỉ mục theo cụm và ràng buộc khóa chính**

Khi bạn tạo bảng có khóa chính, SQL Server sẽ tự động tạo chỉ mục được phân cụm tương ứng bao gồm các cột khóa chính.

In [ ]:
ALTER TABLE production.parts
ADD PRIMARY KEY(part_id)

![indexes-4](.\SQL-Server-Clustered-Index-and-Primary-Key.png)

## **Sử dụng câu lệnh CREATE CLUSTERED INDEX để tạo chỉ mục theo nhóm.**

In [ ]:
CREATE CLUSTERED INDEX ix_parts_id
ON production.parts (part_id)

Nếu bạn mở nút Indexes dưới tên bảng, bạn sẽ thấy tên chỉ mục mới ix\_parts\_id với kiểu Clustered.

![indexes-5](.\SQL-Server-Clustered-Index-example.png)

Khi thực hiện câu lệnh sau, SQL Server duyệt qua chỉ mục (Clustered Index Seek) để xác định vị trí các hàng, nhanh hơn so với việc quét toàn bộ bảng.

In [ ]:
SELECT 
    part_id, 
    part_name
FROM 
    production.parts
WHERE 
    part_id = 5

![indexes-5](.\SQL-Server-Clustered-Index-Clustered-Index-Seek.png)


## **CREATE CLUSTERED INDEX**

Cú pháp để tạo một chỉ mục được phân cụm như sau:

```
CREATE CLUSTERED INDEX index_name
ON schema_name.table_name (column_list)

```

Trong cú pháp này:

- Đầu tiên, chỉ định tên của chỉ mục được nhóm sau mệnh đề CREATE CLUSTERED INDEX. 
- Thứ hai, chỉ định lược đồ và tên bảng mà bạn muốn tạo chỉ mục. 
- Thứ ba, liệt kê một hoặc nhiều cột có trong chỉ mục.

## **Non-clustered indexes**

Chỉ mục không phân tán là một cấu trúc dữ liệu giúp cải thiện tốc độ truy xuất dữ liệu từ các bảng. Không giống như chỉ mục được phân nhóm, chỉ mục không phân nhóm sắp xếp và lưu trữ dữ liệu riêng biệt với các hàng dữ liệu trong bảng. Nó là một bản sao của các cột dữ liệu được chọn từ một bảng với các liên kết đến bảng được liên kết.

Tương tự như chỉ mục được phân nhóm, chỉ mục không phân cụm sử dụng cấu trúc B-tree để tổ chức dữ liệu của nó.

Một bảng có thể có một hoặc nhiều chỉ mục không phân nhóm và mỗi chỉ mục không phân nhóm có thể bao gồm một hoặc nhiều cột của bảng.

![non-indexes-1](.\SQL-Server-nonclustered-index.png)

Bên cạnh việc lưu trữ các giá trị khóa chỉ mục, các nút lá cũng lưu trữ các con trỏ hàng đến các hàng dữ liệu có chứa các giá trị khóa. Các con trỏ hàng này còn được gọi là bộ định vị hàng.

Nếu bảng cơ bản là một bảng nhóm, thì con trỏ hàng là khóa chỉ mục được nhóm. Trong trường hợp bảng bên dưới là một đống, con trỏ hàng trỏ đến hàng của bảng.

## **CREATE NONCLUSTERED INDEX**

```
CREATE NONCLUSTERED INDEX index_name
ON table_name(column_list)

```

Trong cú pháp này:

- Đầu tiên, chỉ định tên của chỉ mục sau mệnh đề CREATE NONCLUSTERED INDEX. Lưu ý rằng từ khóa KHÔNG ĐIỀU CHỈNH là tùy chọn. 
- Thứ hai, chỉ định tên bảng mà bạn muốn tạo chỉ mục và danh sách các cột của bảng đó làm cột khóa chỉ mục.

## **A) Sử dụng câu lệnh CREATE INDEX để tạo chỉ mục không phân nhánh cho một cột**

In [ ]:
SELECT 
    customer_id, 
    city
FROM 
    sales.customers
WHERE 
    city = 'Atwater'

Nếu bạn hiển thị kế hoạch thực thi ước tính, bạn sẽ thấy rằng trình tối ưu hóa truy vấn quét chỉ mục được phân cụm để tìm hàng. Điều này là do bảng sales.customers không có chỉ mục cho cột thành phố.

![non-indexes-2](SQL-Server-CREATE-INDEX-on-one-column-index-scan.png)

Để cải thiện tốc độ của truy vấn này, bạn có thể tạo một chỉ mục mới có tên ix\_customers\_city cho cột thành phố:

In [ ]:
CREATE INDEX ix_customers_city
ON sales.customers(city)

Bây giờ, nếu bạn hiển thị lại kế hoạch thực thi ước tính của truy vấn trên, bạn sẽ thấy rằng trình tối ưu hóa truy vấn sử dụng chỉ mục không phân bổ ix\_customers\_city:

![non-indexes-2](SQL-Server-CREATE-INDEX-one-column-index-seek.png)


## **B) Sử dụng câu lệnh CREATE INDEX để tạo chỉ mục không phân biệt cho nhiều cột**

In [ ]:
SELECT 
    customer_id, 
    first_name, 
    last_name
FROM 
    sales.customers
WHERE 
    last_name = 'Berg' AND 
    first_name = 'Monika'

![non-indexes-4](.\SQL-Server-CREATE-INDEX-on-multiple-columns-index-scan.png)

Trình tối ưu hóa truy vấn quét chỉ mục theo nhóm để xác định khách hàng.

Để tăng tốc độ truy xuất dữ liệu, bạn có thể tạo chỉ mục không phân biệt bao gồm cả cột last\_name và first\_name:

In [ ]:
CREATE INDEX ix_customers_name 
ON sales.customers(last_name, first_name)

In [ ]:
SELECT 
    customer_id, 
    first_name, 
    last_name
FROM 
    sales.customers
WHERE 
    last_name = 'Berg' AND 
    first_name = 'Monika'

![non-indexes-5](.\SQL-Server-CREATE-INDEX-on-multiple-columns-index-seek.png)

Khi bạn tạo một chỉ mục không hợp nhất bao gồm nhiều cột, thứ tự của các cột trong chỉ mục là rất quan trọng. Bạn nên đặt các cột mà bạn thường sử dụng để truy vấn dữ liệu ở đầu danh sách cột.  

Ví dụ, câu lệnh sau đây tìm khách hàng có họ là Albert. Bởi vì last\_name là cột ngoài cùng bên trái trong chỉ mục, trình tối ưu hóa truy vấn có thể tận dụng chỉ mục và sử dụng phương pháp tìm kiếm chỉ mục để tìm kiếm:

In [ ]:
SELECT 
    customer_id, 
    first_name, 
    last_name
FROM 
    sales.customers
WHERE 
    last_name = 'Albert'

![non-indexes-6](.\SQL-Server-CREATE-INDEX-leftmost-column-query.png)

Câu lệnh này tìm khách hàng có tên là Adam. Nó cũng tận dụng chỉ mục ix\_customer\_name. Nhưng nó cần phải quét toàn bộ chỉ mục để tìm kiếm, chậm hơn so với tìm kiếm chỉ mục.

In [ ]:
SELECT 
    customer_id, 
    first_name, 
    last_name
FROM 
    sales.customers
WHERE 
    first_name = 'Adam'

![non-indexes-7](.\SQL-Server-CREATE-INDEX-multiple-columns-not-leftmost-column-index-scan.png)

Do đó, bạn nên đặt các cột mà bạn thường sử dụng để truy vấn dữ liệu ở đầu danh sách cột của chỉ mục.

## **Khi nào thì sử dụng chỉ mục theo cụm?**

Một cột là ứng cử viên tốt nhất cho chỉ mục nhóm nếu một trong những điều sau là đúng:

- Nó được sử dụng trong một số lượng lớn các truy vấn trong mệnh đề WHERE và các phép nối.
- Nó sẽ được sử dụng như một khóa ngoại cho một bảng khác, và cuối cùng, để tham gia.
- Giá trị cột duy nhất.
- Giá trị sẽ ít có khả năng thay đổi.
- Cột đó được sử dụng để truy vấn một dải giá trị. Các toán tử như\>, \<,\> =, \<= hoặc BETWEEN được sử dụng với cột trong mệnh đề WHERE.

Nhưng các chỉ mục được phân nhóm sẽ không tốt nếu cột hoặc các cột

- thường xuyên thay đổi
- là các phím rộng hoặc sự kết hợp của các cột có kích thước phím lớn.

## **Khi nào sử dụng Chỉ mục không phân cụm?**

Một hoặc các cột là ứng cử viên tốt cho các chỉ mục không phân cụm nếu điều sau là đúng:

- Cột hoặc các cột được sử dụng trong mệnh đề WHERE hoặc phép nối.
- Truy vấn sẽ không trả về một tập hợp kết quả lớn.
- Kết hợp chính xác trong mệnh đề WHERE sử dụng toán tử bình đẳng là cần thiết.